# LLMs and Chat Models

In [1]:
# text-davinci-003
from langchain.llms.openai import OpenAI
# gpt-r.5-turbo
from langchain.chat_models import ChatOpenAI
llm = OpenAI()
chat = ChatOpenAI()

# predict just one string, message
a = llm.predict("How many planets are there")
b = chat.predict("How many planets are there")
a,b

('\n\nAs of February 2021, there are eight known planets in our Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are currently 8 confirmed planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

# Predict Messages

In [2]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.1)

In [6]:
# predict list of messages
# message constructors
# 맥락 및 조건 설정 가능, 작성한 것들 memory에 추가, 
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content = "You are a georgraphy expoert. And you only reply in Inalian.",
    ),
    AIMessage(content= "Ciao, mi chiamo Paolo"),
    HumanMessage(
        content = "What is the distance between Mexico and Thailand. Also, what isyour name?",
    ),
]
chat.predict_messages(messages)

'Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 17.000 chilometri.'

# Prompt Template

In [11]:
# 템플릿은 일종의 validation 기능을 해서 쓸대없는 문장을 소비하지 않게 만들어줌
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}."
)
prompt = template.format(country_a = "Mexico",country_b = "Thailand")
chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [14]:
template = ChatPromptTemplate.from_messages(
    [
        ('system',"You are a georgraphy expoert. And you only reply in {language}."),
        ("ai","Ciao, mi chiamo {name}"),
        ("human","What is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)
prompt = template.format_messages(
    language = 'Greek',
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand"

)
chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

# Output Parser and LangChainExpressionLanguage(LCEL)

In [31]:
# LangChain Expression Language
from langchain.schema import BaseOutputParser

# LLM 응답을 변형해야 할 때 사용하기 위해서
class CommaOutputParser(BaseOutputParser):
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
p = CommaOutputParser()
p.parse("Hello,how, are, you")
    

['Hello', 'how', 'are', 'you']

In [32]:
from langchain.prompts import ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ('system', "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else"),
    ("human","{question}")
]
)
prompt = template.format_messages(max_items = 10,question = "What are the planets?")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [33]:
prompt = template.format_messages(max_items = 10, question = "What are the colors")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

- 체인을 이용하면 각각의 predict 결과를 연결해서 사용할 수 있음
- R의 chain 같은 느낌
```
chain1 = template1 | chat | outputparser1
chain2 = template2 | chat | outputparser2
chain_sum = chain1 | chain2 | outputparser_sum
```

In [34]:
# 코드가 너무 김
# 기존 작업을 위해 작성한 코드: Chat model, 파서 생성, 템플릿 생성, message format, predict, 파서 호출, 파싱 
template = ChatPromptTemplate.from_messages([
    ('system', "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else"),
    ("human","{question}")
]
)
# chain 사용
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the poketmons?"
})

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

# Chaining Chains
- [invoke 설명](https://python.langchain.com/docs/expression_language/interface)

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
# Streaming은 모델의 응답이 생성되는걸 보게 해줌, 
chat = ChatOpenAI(
    temperature=0.1,streaming=True,callbacks=[StreamingStdOutCallbackHandler()]
    )


chef_template = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food")
])

chef_chain = chef_template | chat 

In [46]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetereian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    "human","{recipe}"
])
veg_chain = veg_chef_prompt | chat


final_chain = {"recipe": chef_chain} | veg_chain


final_chain.invoke({
    "cuisine":"indian"
})


Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Here's a recipe for a classic Indian dish called Butter Chicken:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt, to taste
- Fresh cilantro, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute, stirring continuously.
4. Add the chicken pieces to the pan and cook until they are browned on all sides.


AIMessageChunk(content="To make a vegetarian version of Butter Chicken, you can replace the chicken with a plant-based protein such as tofu or paneer (Indian cottage cheese). Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter or vegetable oil\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup heavy cream or coconut cream\n- Salt, to taste\n- Fresh cilantro, for garnish\n\nInstructions:\n1. Heat the butter or vegetable oil in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to t

# [Module format](https://python.langchain.com/docs/modules/)
- Model I/O
    - Model Input & Output
    - prompts, language model, output parser


- Retrieval
    - 외부데이터를 모델에 어떻게 적용하느냐

- Chains

- Memory

- Agents


# Fewshot Prompt Template
- prompt template을 디스크에 저장하고 로드할 수 있기 때문에 필요함

In [49]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

t = PromptTemplate(
    template = "What is the capital of {country}",
    input_variables=["country"],
)
t.format(country = "France")

t= PromptTemplate.from_template("What is the capital of {country}")
t.format(country = "France")


'What is the capital of France'

In [56]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

# example_template = """
#     Human: {question}
#     AI: {answer}
# """
# example_prompt =  PromptTemplate.from_template(example_template)

example_prompt =  PromptTemplate.from_template("Human:{question}\nAI:{answer}")

# suffix 는 사용자의 질문
prompt = FewShotPromptTemplate(
    example_prompt= example_prompt,
    examples = examples,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
    )


# chat.predict(prompt.format(country = 'Germany'))
chain = prompt | chat
chain.invoke({
    "country": "Turkey"
})

AI:
        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

'AI:\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro'

# Few Shot Chat Message Prompt Template

In [63]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
        ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example prompt는 example과 key값이 일치해야함
example_prompt =  ChatPromptTemplate.from_messages(
    [
    ("human", "What do you know about {country}?"),
    ("ai","{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt= example_prompt,
    examples = examples,
    )

final_prompt = ChatPromptTemplate.from_messages(
    [
    ("system"," You are a geography expert, you gives short answers"),
    example_prompt,
    ("human","What do you know about {country}?"),
    ],
)
# chat.predict(prompt.format(country = 'Germany'))
chain = final_prompt | chat
chain.invoke({
    "country": "THailand"
})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum Soup
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum Soup\n        Currency: Thai Baht\n        ')